In [25]:
# Import the pandas library.
import pandas
import seaborn as sns
import numpy as np
import haversine as hs

### Read the CSV file

In [26]:
airports = pandas.read_csv("airports_cleaned_dataset.csv")

In [27]:
airports.head()

,ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Offset,DST,Timezone
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10.0,U,Pacific/Port_Moresby
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10.0,U,Pacific/Port_Moresby
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10.0,U,Pacific/Port_Moresby
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10.0,U,Pacific/Port_Moresby
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10.0,U,Pacific/Port_Moresby


### Create Coordination for Airports Location

In [28]:
airports['Coordination'] = list(zip(airports.Latitude, airports.Longitude))
airports

,ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Offset,DST,Timezone,Coordination
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10.0,U,Pacific/Port_Moresby,"(-6.081689834590001, 145.391998291)"
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10.0,U,Pacific/Port_Moresby,"(-5.20707988739, 145.789001465)"
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10.0,U,Pacific/Port_Moresby,"(-5.826789855957031, 144.29600524902344)"
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10.0,U,Pacific/Port_Moresby,"(-6.569803, 146.725977)"
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10.0,U,Pacific/Port_Moresby,"(-9.44338035583496, 147.22000122070312)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5510,9935,Kualanamu International Airport,Medan,Indonesia,KNO,WIMM,3.642222,98.885278,23,7.0,N,Asia/Jakarta,"(3.642222, 98.885278)"
5511,10114,Chichen Itza International Airport,Chichen Itza,Mexico,CZA,MMCT,20.641300,-88.446198,102,-6.0,S,America/Mexico_City,"(20.6413002014, -88.4461975098)"
5512,10135,San Cristobal de las Casas Airport,San Cristobal de las Casas,Mexico,SZT,MMSC,16.690300,-92.530098,7707,-6.0,N,America/Mexico_City,"(16.690299987793, -92.530097961426)"
5513,10156,Yeysk Airport,Eysk,Russia,EIK,URKE,46.680000,38.210000,60,3.0,N,Europe/Moscow,"(46.68, 38.21)"


In [29]:
airports.columns

Index(['ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude',
       'Longitude', 'Altitude', 'Offset', 'DST', 'Timezone', 'Coordination'],
      dtype='object')

### Create a data frame for Airports around Malaysia

In [35]:
airports_Malaysia = airports[airports['Country'] == 'Malaysia']
  
airports_Malaysia.head()

,ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Offset,DST,Timezone,Coordination
2479,3262,Bintulu Airport,Bintulu,Malaysia,BTU,WBGB,3.12385,113.019997,74,8.0,N,Asia/Kuala_Lumpur,"(3.12385010719, 113.019996643)"
2480,3263,Kuching International Airport,Kuching,Malaysia,KCH,WBGG,1.48470,110.347000,89,8.0,N,Asia/Kuala_Lumpur,"(1.4846999645233154, 110.34700012207033)"
2481,3264,Limbang Airport,Limbang,Malaysia,LMN,WBGJ,4.80830,115.010002,14,8.0,N,Asia/Kuala_Lumpur,"(4.808300018310547, 115.01000213623048)"
2482,3265,Marudi Airport,Marudi,Malaysia,MUR,WBGM,4.17898,114.329002,103,8.0,N,Asia/Kuala_Lumpur,"(4.178979873657227, 114.3290023803711)"
2483,3266,Miri Airport,Miri,Malaysia,MYY,WBGR,4.32201,113.987000,59,8.0,N,Asia/Kuala_Lumpur,"(4.322010040283203, 113.98699951171876)"


### Defining a function to calculate distance between two locations

In [38]:
# Location 1 : Airport A
# Location 2 : Airport B

def distance_from(loc1,loc2): 
    dist=hs.haversine(loc1,loc2)
    return round(dist,2)

In [39]:
for _,row in airports_Malaysia.iterrows():
    airports_Malaysia[row.Name]=airports_Malaysia['Coordination'].apply(lambda x: distance_from(row.Coordination,x))

<ipython-input-39-a8baabed99da>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airports_Malaysia[row.Name]=airports_Malaysia['Coordination'].apply(lambda x: distance_from(row.Coordination,x))


In [41]:
airports_Malaysia

,ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Offset,...,Long Lellang Airport,Long Seridan Airport,Mukah Airport,Bakalalan Airport,Lawas Airport,Bario Airport,Tomanggong Airport,Kudat Airport,Pulau Pangkor Airport,Long Akah Airport
2479,3262,Bintulu Airport,Bintulu,Malaysia,BTU,WBGB,3.12385,113.019997,74,8.0,...,239.20,244.02,107.14,303.43,327.06,281.24,674.50,597.51,1388.97,196.71
2480,3263,Kuching International Airport,Kuching,Malaysia,KCH,WBGG,1.48470,110.347000,89,8.0,...,575.77,590.76,249.13,647.54,674.98,622.48,1019.97,939.70,1130.05,532.55
2481,3264,Limbang Airport,Limbang,Malaysia,LMN,WBGJ,4.80830,115.010002,14,8.0,...,155.09,93.65,387.79,114.67,44.33,130.30,409.36,309.93,1603.72,169.59
2482,3265,Marudi Airport,Marudi,Malaysia,MUR,WBGM,4.17898,114.329002,103,8.0,...,124.43,83.37,286.92,144.77,140.92,136.83,498.55,412.35,1527.68,109.96
2483,3266,Miri Airport,Miri,Malaysia,MYY,WBGR,4.32201,113.987000,59,8.0,...,163.71,124.32,263.75,184.98,168.06,177.94,531.24,427.78,1489.63,143.92
2484,3267,Sibu Airport,Sibu,Malaysia,SBW,WBGS,2.26160,111.985001,122,8.0,...,374.81,389.57,72.47,446.04,476.52,421.10,818.47,746.68,1288.08,331.51
2485,3268,Lahad Datu Airport,Lahad Datu,Malaysia,LDU,WBKD,5.03225,118.323997,45,8.0,...,394.54,381.77,731.82,322.22,323.68,346.89,55.32,266.94,1971.45,437.38
2486,3269,Kota Kinabalu International Airport,Kota Kinabalu,Malaysia,BKI,WBKK,5.93721,116.051003,10,8.0,...,296.92,245.54,554.39,223.51,140.37,253.06,294.49,139.74,1726.68,325.18
2487,3270,Labuan Airport,Labuan,Malaysia,LBU,WBKL,5.30068,115.250000,101,8.0,...,209.28,149.95,440.99,153.05,53.17,176.06,377.43,251.53,1632.74,228.41
2488,3271,Tawau Airport,Tawau,Malaysia,TWU,WBKW,4.32016,118.127998,57,8.0,...,344.75,343.61,689.32,281.02,307.17,300.97,133.90,322.76,1948.77,388.07


### Delete columns 

In [47]:
new_airports_malaysia = airports_Malaysia.drop(['Country', 'ICAO','Latitude','Longitude','Altitude','Offset','DST','Timezone'], axis = 1)
new_airports_malaysia

,ID,Name,City,IATA,Coordination,Bintulu Airport,Kuching International Airport,Limbang Airport,Marudi Airport,Miri Airport,...,Long Lellang Airport,Long Seridan Airport,Mukah Airport,Bakalalan Airport,Lawas Airport,Bario Airport,Tomanggong Airport,Kudat Airport,Pulau Pangkor Airport,Long Akah Airport
2479,3262,Bintulu Airport,Bintulu,BTU,"(3.12385010719, 113.019996643)",0.00,348.45,289.50,186.72,171.06,...,239.20,244.02,107.14,303.43,327.06,281.24,674.50,597.51,1388.97,196.71
2480,3263,Kuching International Airport,Kuching,KCH,"(1.4846999645233154, 110.34700012207033)",348.45,0.00,636.03,534.13,512.74,...,575.77,590.76,249.13,647.54,674.98,622.48,1019.97,939.70,1130.05,532.55
2481,3264,Limbang Airport,Limbang,LMN,"(4.808300018310547, 115.01000213623048)",289.50,636.03,0.00,102.94,125.62,...,155.09,93.65,387.79,114.67,44.33,130.30,409.36,309.93,1603.72,169.59
2482,3265,Marudi Airport,Marudi,MUR,"(4.178979873657227, 114.3290023803711)",186.72,534.13,102.94,0.00,41.12,...,124.43,83.37,286.92,144.77,140.92,136.83,498.55,412.35,1527.68,109.96
2483,3266,Miri Airport,Miri,MYY,"(4.322010040283203, 113.98699951171876)",171.06,512.74,125.62,41.12,0.00,...,163.71,124.32,263.75,184.98,168.06,177.94,531.24,427.78,1489.63,143.92
2484,3267,Sibu Airport,Sibu,SBW,"(2.2616000175476074, 111.98500061035156)",149.69,201.50,439.19,336.41,319.18,...,374.81,389.57,72.47,446.04,476.52,421.10,818.47,746.68,1288.08,331.51
2485,3268,Lahad Datu Airport,Lahad Datu,LDU,"(5.032249927520752, 118.3239974975586)",625.36,969.32,367.98,452.84,487.09,...,394.54,381.77,731.82,322.22,323.68,346.89,55.32,266.94,1971.45,437.38
2486,3269,Kota Kinabalu International Airport,Kota Kinabalu,BKI,"(5.9372100830078125, 116.0510025024414)",459.05,803.44,170.41,273.13,290.70,...,296.92,245.54,554.39,223.51,140.37,253.06,294.49,139.74,1726.68,325.18
2487,3270,Labuan Airport,Labuan,LBU,"(5.300680160522461, 115.25)",346.03,690.02,60.86,161.16,177.28,...,209.28,149.95,440.99,153.05,53.17,176.06,377.43,251.53,1632.74,228.41
2488,3271,Tawau Airport,Tawau,TWU,"(4.320159912109375, 118.12799835205078)",582.18,919.74,349.84,421.56,459.15,...,344.75,343.61,689.32,281.02,307.17,300.97,133.90,322.76,1948.77,388.07


### Convert Malaysia Airports Data Frame to CSV

In [50]:
new_airports_malaysia.to_csv('airlines_info_malaysia.csv', index=False)